# LangChain Expression Language (LCEL)

In [36]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [37]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [38]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [39]:
#!pip install pydantic==1.10.8

In [40]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [41]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [42]:
chain = prompt | model | output_parser

In [43]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\nBecause they already have bear feet!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [44]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [45]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [46]:
retriever.get_relevant_documents("where did harrison work?")

[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [47]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [48]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [49]:
from langchain.schema.runnable import RunnableMap

In [50]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [51]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [52]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [53]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [54]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [55]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [56]:
runnable = prompt | model

In [57]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 64, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-b422fdcd-9c96-4011-a86d-4fc72b34f26a-0')

In [58]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [59]:
model = model.bind(functions=functions)

In [60]:
runnable = prompt | model

In [61]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"New England Patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-046c4c5f-30f3-4e55-bd9c-762ba0e7f111-0')

## Fallbacks

In [62]:
from langchain.llms import OpenAI
import json

**Note**: Due to the deprication of OpenAI's model `text-davinci-001` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

In [63]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

/var/folders/kv/h05fnf25189cjzsw9tvnfgm40000gn/T/ipykernel_21678/2498289737.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  simple_model = OpenAI(


In [64]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [65]:
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [66]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

In [ ]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [ ]:
chain.invoke(challenge)

In [ ]:
final_chain = simple_chain.with_fallbacks([chain])

In [ ]:
final_chain.invoke(challenge)

## Interface

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

In [ ]:
for t in chain.stream({"topic": "bears"}):
    print(t)

In [ ]:
response = await chain.ainvoke({"topic": "bears"})
response